In [1]:
from keras.models import Sequential
from keras.layers import Dense

import numpy as np
# fix random seed for reproducibility
np.random.seed(42)

import pandas as pd

Using TensorFlow backend.


In [2]:
#Folder for the dataset
datasetFolder = '/home/carnd/dbpedia2016/dataset/'

#Number of files
numberOfFiles = 511

#Test split
testSplit=0.25

In [3]:
def load_data(datasetFolder, datasetXFile, datasetYFile):
    #print('Loading {} & {}'.format(datasetXFile, datasetYFile))
    # load file
    with open(datasetFolder + datasetXFile, "r") as f:
        head = f.readline()
        cols = head.split(',')
        numberOfCols = len(cols)
        #print(numberOfCols)
        numberOfRows=0
        for line in f:
            numberOfRows+=1
        f.close()
    
    #print('{} x {}'.format(numberOfRows,numberOfCols))
    dataX = np.zeros([numberOfRows,numberOfCols-1],np.float32)
    
    with open(datasetFolder + datasetXFile, "r") as f:
        head = f.readline()
        rowCounter=0
        for line in f:
            row=line.split(', ')
            for i in range(1, len(row)):
                if(int(row[i])<=0):
                    continue;
                dataX[rowCounter][int(row[i])-1]=1.0
            rowCounter+=1
        f.close()
        
    dataY=pd.read_csv(datasetFolder + datasetYFile)

    # delete the id fields
    #del dataX['id']
    del dataY['id']

    return dataX, dataY.as_matrix()

In [4]:
dataX, dataY = load_data(datasetFolder,'datasetX_1.csv', 'datasetY_1.csv')

In [5]:
print(dataX.shape)
print(dataX[0:5])

(6188, 6054)
[[ 0.  1.  1. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  1.  0. ...,  0.  0.  0.]
 [ 0.  1.  1. ...,  0.  0.  0.]]


In [6]:
print(dataY.shape)
print(dataY[0:5])

(6188, 539)
[[0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [1 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]]


In [7]:
print("Input Features: {}".format(dataX.shape[1]))
print("Output Classes: {}".format(dataY.shape[1]))

Input Features: 6054
Output Classes: 539


In [8]:
# create model
logisticRegression = Sequential(name='Simple Logistic Regression')
logisticRegression.add(Dense(dataY.shape[1], input_dim=dataX.shape[1], activation='sigmoid', init='glorot_uniform'))

simpleModel = Sequential(name='2 Fully Connected Layers')
simpleModel.add(Dense(1024, input_dim=dataX.shape[1], activation='relu', init='glorot_uniform'))
simpleModel.add(Dense(dataY.shape[1], activation='sigmoid', init='glorot_uniform'))

deepModel = Sequential(name='Deep Model (5 Dense Layers)')
deepModel.add(Dense(2048, input_dim=dataX.shape[1], activation='relu', init='glorot_uniform'))
deepModel.add(Dense(1024, activation='relu', init='glorot_uniform'))
deepModel.add(Dense(768, activation='relu', init='glorot_uniform'))
deepModel.add(Dense(768, activation='relu', init='glorot_uniform'))
deepModel.add(Dense(dataY.shape[1], activation='sigmoid', init='glorot_uniform'))

models = [logisticRegression, simpleModel, deepModel]

In [9]:
# Compile model
import keras.backend as K

def count_predictions(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    return true_positives, predicted_positives, possible_positives

def f1score(y_true, y_pred):
    true_positives, predicted_positives, possible_positives = count_predictions(y_true, y_pred)
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1score = 2.0 * precision * recall / (precision+recall+ K.epsilon())
    return f1score

def fBetaScore(y_true, y_pred, beta):
    true_positives, predicted_positives, possible_positives = count_predictions(y_true, y_pred)
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1score = (1+(beta*beta)) * precision * recall / ((beta*beta*precision)+recall+ K.epsilon())
    return f1score

for model in models:
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[f1score])

In [10]:
def fit_data(model, dataX, dataY):
    # Fit the model
    #model.fit(dataX, dataY, nb_epoch=5, verbose=2, batch_size=256)
    return model.train_on_batch(dataX, dataY)

In [11]:
def countPredictions(y_true, y_pred):
    true_positives = np.sum(np.round(y_pred*y_true))
    predicted_positives = np.sum(np.round(y_pred))
    possible_positives = np.sum(y_true)
    return true_positives, predicted_positives, possible_positives

In [12]:
#Randomize the list of numbers so we can split train and test dataset
listOfFiles=list(range(1,numberOfFiles+1))
import random
random.shuffle(listOfFiles)
splitIndex=int((1-testSplit)*numberOfFiles)

numberOfEons = 5
for eon in range(0, numberOfEons):
    print('\n***********************************\n {}/{}'.format(eon+1, numberOfEons))
    for trainIndex in range(0,splitIndex):
        print('==========================================')
        print('Learning for file {} / {} : datasetX/Y_{}...'.format(trainIndex+1, splitIndex, listOfFiles[trainIndex]))
        dataX, dataY = load_data(datasetFolder,'datasetX_{}.csv'.format(listOfFiles[trainIndex]), 'datasetY_{}.csv'.format(listOfFiles[trainIndex]))
        for model in models:
            #print('Model = {}'.format(model.name))
            loss, f1score=fit_data(model,dataX, dataY)
            print('   Model = {} \t loss = {:.4f} \t f1-score = {:.4f}'.format(model.name, loss, f1score))
            
        
    counts = {} 
    for model in models:
        counts[model.name] = {'true_positives':0, 'predicted_positives':0, 'possible_positives':0}
    
    print('==========================================\n')
    for testIndex in range(splitIndex, numberOfFiles):
        dataX, dataY = load_data(datasetFolder,'datasetX_{}.csv'.format(listOfFiles[testIndex]), 'datasetY_{}.csv'.format(listOfFiles[testIndex]))
        print ('Testing for file {} / {} : datasetX/Y_{}'.format(testIndex+1, numberOfFiles, listOfFiles[testIndex]))
        for model in models:
            predY=model.predict_on_batch(dataX)
            true_positives, predicted_positives, possible_positives = countPredictions(dataY, predY)
            counts[model.name]['true_positives'] += true_positives
            counts[model.name]['predicted_positives'] += predicted_positives
            counts[model.name]['possible_positives'] += possible_positives
    print('==========================================\n')
    
    
    print('{}.  {}/{} - Testing score:'.format(eon+1, eon+1, numberOfEons))
    for model in models:
        count = counts[model.name]
        precision = (count['true_positives'])/(count['predicted_positives']+0.0001)
        recall = (count['true_positives'])/(count['possible_positives']+0.0001)
        f1score = 2.0 * precision * recall / (precision+recall+0.0001)
        print(' - Model = {} \t f1-score = {:.4f}\t precision = {:.4f} \t recall = {:.4f}'.format(model.name, f1score, precision, recall))
    print('==========================================\n')


***********************************
 1/5
Learning for file 1 / 383 : datasetX/Y_347...
   Model = Simple Logistic Regression 	 loss = 0.6933 	 f1-score = 0.0121
   Model = 2 Fully Connected Layers 	 loss = 0.6928 	 f1-score = 0.0124
   Model = Deep Model (5 Dense Layers) 	 loss = 0.6932 	 f1-score = 0.0125
Learning for file 2 / 383 : datasetX/Y_279...
   Model = Simple Logistic Regression 	 loss = 0.6912 	 f1-score = 0.0128
   Model = 2 Fully Connected Layers 	 loss = 0.6857 	 f1-score = 0.0125
   Model = Deep Model (5 Dense Layers) 	 loss = 0.6851 	 f1-score = 0.0149
Learning for file 3 / 383 : datasetX/Y_77...
   Model = Simple Logistic Regression 	 loss = 0.6888 	 f1-score = 0.0144
   Model = 2 Fully Connected Layers 	 loss = 0.6771 	 f1-score = 0.0128
   Model = Deep Model (5 Dense Layers) 	 loss = 0.6602 	 f1-score = 0.0207
Learning for file 4 / 383 : datasetX/Y_82...
   Model = Simple Logistic Regression 	 loss = 0.6863 	 f1-score = 0.0159
   Model = 2 Fully Connected Layers 	 l

==================================================
# all3_1x15
==================================================
1. Eon 1/5 - Testing score: 
 - Model = Simple Logistic Regression 	 f1-score = 0.7228	 precision = 0.9145 	 recall = 0.5977
 - Model = 2 Fully Connected Layers 	 f1-score = 0.8039	 precision = 0.9196 	 recall = 0.7142
 - Model = Deep Model (5 Dense Layers) 	 f1-score = 0.8867	 precision = 0.9334 	 recall = 0.8444
2. Eon 2/5 - Testing score: 
 - Model = Simple Logistic Regression 	 f1-score = 0.7795	 precision = 0.9135 	 recall = 0.6798
 - Model = 2 Fully Connected Layers 	 f1-score = 0.8570	 precision = 0.9295 	 recall = 0.7950
 - Model = Deep Model (5 Dense Layers) 	 f1-score = 0.8983	 precision = 0.9330 	 recall = 0.8662
3. Eon 3/5 - Testing score:
 - Model = Simple Logistic Regression 	 f1-score = 0.8022	 precision = 0.9159 	 recall = 0.7137
 - Model = 2 Fully Connected Layers 	 f1-score = 0.8747	 precision = 0.9328 	 recall = 0.8236
 - Model = Deep Model (5 Dense Layers) 	 f1-score = 0.9005	 precision = 0.9339 	 recall = 0.8694
4. Eon 4/5 - Testing score:
 - Model = Simple Logistic Regression 	 f1-score = 0.8160	 precision = 0.9165 	 recall = 0.7354
 - Model = 2 Fully Connected Layers 	 f1-score = 0.8826	 precision = 0.9342 	 recall = 0.8364
 - Model = Deep Model (5 Dense Layers) 	 f1-score = 0.9014	 precision = 0.9339 	 recall = 0.8711
5. Eon 5/5 - Testing score:
 - Model = Simple Logistic Regression 	 f1-score = 0.8251	 precision = 0.9173 	 recall = 0.7498
 - Model = 2 Fully Connected Layers 	 f1-score = 0.8868	 precision = 0.9352 	 recall = 0.8432
 - Model = Deep Model (5 Dense Layers) 	 f1-score = 0.9015	 precision = 0.9343 	 recall = 0.8709